In [1]:
from osgeo import gdal
import os
import matplotlib.pyplot as plt


os.chdir(r'D:\Projects\VsCode\Python\img_processing_system\osgeopy-data\Landsat\Washington') 
band1_fn = 'p047r027_7t20000730_z10_nn10.tif' 

in_ds = gdal.Open(band1_fn)
in_band = in_ds.GetRasterBand(1)
in_band_arr = in_band.ReadAsArray(1000, 1000, 3000, 3000)

gtiff_driver = gdal.GetDriverByName('GTiff') 
res_ds = gtiff_driver.Create("my_img.tif", 3000,3000, bands = 1)
res_ds.GetRasterBand(1).WriteArray(in_band_arr)

res_ds.FlushCache() 

del res_ds



In [2]:
from osgeo import gdal
import os
os.chdir(r'D:\Projects\VsCode\Python\img_processing_system\osgeopy-data\Landsat\Washington') 

in_ds = gdal.Open("p047r027_7t20000730_z10_nn10.tif")

row_len = in_ds.GetRasterBand(1).YSize * 2
col_len = in_ds.GetRasterBand(1).XSize * 2

gtiff_driver = gdal.GetDriverByName('GTiff')
res_ds = gtiff_driver.Create("p047r027_7t20000730_z10_nn10.tif_more_pixels.tif", col_len, row_len, bands = 1)
res_ds.SetProjection(in_ds.GetProjection()) 

geotransform = list(in_ds.GetGeoTransform()) 
geotransform [1] /= 2 
geotransform [5] /= 2 
res_ds.SetGeoTransform(geotransform)

# ReadAsArray为什么能读这么多？没看懂
# in_ds.GetRasterBand(1)的长和宽都只有col_len，row_len的一半，buf_xsize是指在输出的array的长度
# 这个WriteArray应该是做了一些处理
res_ds.GetRasterBand(1).WriteArray(in_ds.GetRasterBand(1).ReadAsArray(buf_xsize=col_len, buf_ysize=row_len))

res_ds.FlushCache() 

del res_ds

In [3]:
from osgeo import gdal

# 输入和输出文件名
input_file = "my_img.tif"
output_file = "my_img_resampled.tif"

# 获取原始数据集
src_ds = gdal.Open(input_file)
src_proj = src_ds.GetProjection()
src_gt = src_ds.GetGeoTransform()
src_xsize = src_ds.RasterXSize
src_ysize = src_ds.RasterYSize

# 目标大小为原来的2倍（像素数量4倍）
dst_xsize = src_xsize * 2
dst_ysize = src_ysize * 2

# 使用gdal.Warp进行重采样
gdal.Warp(
    output_file,
    src_ds,
    width=dst_xsize,
    height=dst_ysize,
    resampleAlg='bilinear',  # 可选: 'nearest', 'bilinear', 'cubic'等
    outputBounds=None,
    dstSRS=src_proj
)

TypeError: in method 'wrapper_GDALWarpDestName', argument 4 of type 'GDALWarpAppOptions *'